In [ ]:
import os 

In [ ]:
os.getcwd()

In [ ]:
os.chdir("..")

In [ ]:
os.getcwd()

# constants

In [ ]:
from dataclasses import dataclass 
from churn_modelling.utils import load_yaml 


CONFIG = load_yaml("config/config.yaml")

@dataclass 
class ModelPredictionConstants: 
    ROOT_DIR = CONFIG.MODEL.PREDICTION.ROOT_DIR
    FILE_EXT = CONFIG.MODEL.PREDICTION.FILE_EXT 
    BUCKET = os.getenv("S3_BUCKET")
    OBJECT = os.getenv("S3_BUCKET_PREDICTION_OBJECT")

In [ ]:
print(f"ROOT_DIR:{ModelPredictionConstants.ROOT_DIR}")
print(f"FILE_EXT:{ModelPredictionConstants.FILE_EXT}")
print(f"BUCKET:{ModelPredictionConstants.BUCKET}")
print(f"OBJECT:{ModelPredictionConstants.OBJECT}")

# entity

In [ ]:
from dataclasses import dataclass 
from typing import ClassVar 
from pathlib import Path 
import os 


@dataclass 
class ModelPrediction:
    ROOT_DIR_PATH:ClassVar[Path]
    FILE_PATH:ClassVar[Path]
    BUCKET:ClassVar[str]
    OBJECT:ClassVar[str]

# configuration

In [ ]:
from dataclasses import dataclass 
from pathlib import Path 
import os 


@dataclass 
class ModelPredictionConfig:
    ROOT_DIR_PATH = Path(ModelPredictionConstants.ROOT_DIR)
    FILE_PATH = Path(os.path.join(ROOT_DIR_PATH, ModelPredictionConstants.FILE_EXT))
    BUCKET = ModelPredictionConstants.BUCKET
    OBJECT = ModelPredictionConstants.OBJECT

In [ ]:
print(f"ROOT_DIR_PATH:{ModelPredictionConfig.ROOT_DIR_PATH}")
print(f"FILE_PATH:{ModelPredictionConfig.FILE_PATH}")
print(f"BUCKET:{ModelPredictionConfig.BUCKET}")
print(f"OBJECT:{ModelPredictionConfig.OBJECT}")

In [ ]:
print(f"ROOT_DIR_PATH:{type(ModelPredictionConfig.ROOT_DIR_PATH)}")
print(f"FILE_PATH:{type(ModelPredictionConfig.FILE_PATH)}")
print(f"BUCKET:{type(ModelPredictionConfig.BUCKET)}")
print(f"OBJECT:{type(ModelPredictionConfig.OBJECT)}")

In [ ]:
os.path.split("artifacts/model/predictions/.json")

# components

In [ ]:
from churn_modelling.utils import dump_json, load_json, create_dirs
from churn_modelling.exception import CustomException 
from sklearn.compose import ColumnTransformer
from churn_modelling.logger import logging
from churn_modelling.cloud import S3_Cloud  
from skorch import NeuralNetClassifier
from dataclasses import dataclass 
from dotenv import load_dotenv
from datetime import datetime
from pathlib import Path 
import numpy as np
import pandas as pd 
import sys, os 


@dataclass 
class ModelPredictionComponents:
    model_prediction_config:ModelPrediction 

    def predict(self):
        try:
            logging.info("In predict")

            # transform data and perform prediction
            columns = [name.split('__')[1] for name in self.preprocessor.get_feature_names_out()]
            transformed_data = self.preprocessor.transform(self.data)
            model_input = pd.DataFrame(transformed_data, columns=columns).astype(np.float32)
            self.prediction = self.model.predict(model_input)[0]

            logging.info("Out predict")
        except Exception as e:
            logging.exception(e)
            raise CustomException(e, sys) 
        
    def save_outputs(self, time:datetime):
        try:
            logging.info("In save_outputs")
            time_stamp = datetime.now().strftime("%d_%m_%Y")
            dir_path, file_name = os.path.split(self.model_prediction_config.FILE_PATH)
            self.output_file_path = Path(os.path.join(dir_path, time_stamp + file_name))
            
            # load prevous predictions from cloud if not available in local 
            if not os.path.exists(self.output_file_path):
                self.pull_from_cloud()
            
            output = {
                time:{
                    "input":self.data.values.tolist(),
                    "output":self.prediction
                }
            }

            # load prevoiusly saved data if available 
            if os.path.exists(self.output_file_path):
                output.update(load_json(self.output_file_path))

            # save data into file 
            dump_json(output, self.output_file_path)
            logging.info(f"saved outputs at {{{self.output_file_path}}}")

            logging.info("Out save_outputs")
        except Exception as e:
            logging.exception(e)
            raise CustomException(e, sys) 
        
    def push_to_cloud(self):
        try:
            logging.info("In push_to_cloud") 
            load_dotenv()

            cloud=S3_Cloud(
                bucket=os.getenv("S3_BUCKET"),
                object_name=os.getenv("S3_BUCKET_PREDICTION_OBJECT")
            )
            status = cloud.upload_file(self.output_file_path)
            logging.info(f"push status {{{status}}}")
            logging.info("Out push_to_cloud")
        except Exception as e:
            raise CustomException(e, sys)
        
    def pull_from_cloud(self) -> bool:
        try:
            logging.info("In pull_from_cloud") 
            load_dotenv()
            
            cloud=S3_Cloud(
                bucket=self.model_prediction_config.BUCKET,
                object_name=self.model_prediction_config.OBJECT
            )
            status = cloud.download_file(self.output_file_path)
            logging.info(f"push status {{{status}}}")
            logging.info("Out push_to_cloud")
        except:
            pass 
        
    def main(self, model:NeuralNetClassifier, preprocessor:ColumnTransformer, data:pd.DataFrame, time:datetime) -> np.int64:
        # create required directories
        create_dirs(self.model_prediction_config.ROOT_DIR_PATH)

        self.model = model
        self.preprocessor = preprocessor
        self.data = data
        self.predict()
        self.save_outputs(time)
        self.push_to_cloud()

        return self.prediction 

# pipeline

In [ ]:
from sklearn.compose import ColumnTransformer
from skorch import NeuralNetClassifier
from dataclasses import dataclass 
import pandas as pd 
import numpy as np 


@dataclass 
class PredictionPipeline:
    def run(self, model:NeuralNetClassifier, preprocessor:ColumnTransformer, data:pd.DataFrame) -> np.int64:
        obj = ModelPredictionComponents(ModelPredictionConfig)
        return obj.main(model, preprocessor, data)